In [1]:
from tensorflow.keras.layers import Dense, Embedding, Input, Flatten, Multiply, Concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras import Model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.keras.initializers import RandomNormal

from sklearn.metrics import mean_squared_error
from sklearn.metrics import ndcg_score

from recommenders.datasets import movielens
from recommenders.datasets.python_splitters import python_chrono_split
import pandas as pd
import numpy as np


In [2]:
data = movielens.load_pandas_df(size='1m')

100%|█████████████████████████████████████████████████████████████████| 5.78k/5.78k [00:00<00:00, 18.6kKB/s]


In [3]:
train_data, test_data = python_chrono_split(data, 0.8)

In [4]:
train_data.columns = ['user_id','product_id','rating','timestamp']
test_data.columns = ['user_id','product_id','rating','timestamp']

In [5]:
def build_model(num_users, num_items, mf_output_dim,
          embeddings_init, reg_parameter=0, layers=[10],
          user_weights=None, product_weights=None):

    user_input = Input((1,), dtype='int64',name='user_input')
    product_input = Input((1,), dtype= 'int64',name='product_input')

    if user_weights and product_weights:
        mf_user_embeddings = Embedding(num_users, mf_output_dim, input_length=1, weights=[user_weights],
                                       embeddings_regularizer=l2(reg_parameter))
        mf_product_embeddings = Embedding(num_items, mf_output_dim, input_length=1, weights=[product_weights],
                                          embeddings_regularizer=l2(reg_parameter))

        mlp_user_embeddings = Embedding(num_users, mf_output_dim, input_length=1, weights=[user_weights],
                                       embeddings_regularizer=l2(reg_parameter))
        mlp_product_embeddings = Embedding(num_items, mf_output_dim, input_length=1, weights=[product_weights],
                                          embeddings_regularizer=l2(reg_parameter))



    else:
        mf_user_embeddings = Embedding(num_users, mf_output_dim, input_length=1,
                                       embeddings_initializer = embeddings_init, embeddings_regularizer=l2(reg_parameter))
        mf_product_embeddings = Embedding(num_items, mf_output_dim, input_length=1,
                                          embeddings_initializer = embeddings_init, embeddings_regularizer=l2(reg_parameter))

        mlp_user_embeddings = Embedding(num_users, mf_output_dim, input_length=1,embeddings_initializer=embeddings_init,
                                        embeddings_regularizer=l2(reg_parameter))
        mlp_product_embeddings = Embedding(num_items, mf_output_dim, input_length=1,embeddings_initializer=embeddings_init,
                                          embeddings_regularizer=l2(reg_parameter))


    ## Matrix Factorisation
    mf_user_latent = Flatten()(mf_user_embeddings(user_input))
    mf_product_latent = Flatten()(mf_product_embeddings(product_input))

    mf_latent = Multiply()([mf_user_latent, mf_product_latent])


    ## MLP
    mlp_user_latent = Flatten()(mlp_user_embeddings(user_input))
    mlp_product_latent = Flatten()(mlp_product_embeddings(product_input))

    mlp_latent = Concatenate()([mlp_user_latent, mlp_product_latent])

    for i in range(1, len(layers)):
        layer = Dense(layers[i], activation='relu', kernel_regularizer = l2(reg_parameter))
        mlp_latent = layer(mlp_latent)

    final_vector = Concatenate()([mf_latent, mlp_latent])

    prediction = Dense(1, activation='linear')(final_vector)

    model = Model(inputs=[user_input, product_input], outputs=prediction)
    return model








In [6]:
unique_prods = set(pd.concat([train_data['product_id'],test_data['product_id']]))

In [7]:
user2id = {v:k for k,v in enumerate(train_data['user_id'].unique())}
id2user = {v:k for k,v in user2id.items()}

prod2id = {v:k for k,v in enumerate(unique_prods)}
id2prod = {v:k for k,v in prod2id.items()}




In [8]:
max(id2prod.keys())

3705

In [9]:
len(user2id), len(prod2id)

(6040, 3706)

In [10]:
train_data['new_user_id'] = train_data['user_id'].apply(lambda x : user2id[x])
test_data['new_user_id'] = test_data['user_id'].apply(lambda x : user2id[x])

In [11]:
train_data['new_prod_id'] = train_data['product_id'].apply(lambda x : prod2id[x])
test_data['new_prod_id'] = test_data['product_id'].apply(lambda x : prod2id[x])


In [12]:
train_data['rating'] = train_data['rating'].apply(lambda x : float(x))

In [13]:
test_users = test_data['new_user_id'].values
test_products = test_data['new_prod_id'].values

In [14]:
users = train_data['new_user_id'].values
products = train_data['new_prod_id'].values
ratings = train_data['rating'].values

In [15]:
def calculate_metrics(user_df, top_k=10, threshold=4):
    
    user_df = user_df.sort_values(by='predictions', ascending=False)
    
    true_relevant = (user_df['rating'] >= threshold).sum()
    recommended = (user_df['predictions'][:top_k] >= threshold).sum()
    
    numerator = sum((true>=4) and (pred>=4) for (true,pred) in zip(user_df['rating'][:top_k], user_df['predictions'][:top_k]))
    
    prec = numerator / recommended if recommended>0 else 0
    recall = numerator / true_relevant if true_relevant>0 else 0
    
    ndcg = ndcg_score([user_df['rating'].values], [user_df['predictions'].values], k=top_k)
    
    return pd.Series({'precision':prec,'recall':recall,'ndcg':ndcg})
    
    
    
    
    
    

In [16]:
embeddings_factor = [8,16,32,64,100]
n_layers = [128],[128,64], [128,64,32]
output_dir = 'models/'

metrics_list = []
for output_dim in embeddings_factor:
    for layer in n_layers:
        model = build_model(len(user2id),len(prod2id),output_dim, RandomNormal(0,0.01),reg_parameter=0.01,layers=layer)
        model.compile('adam',loss=MeanSquaredError(), metrics = [RootMeanSquaredError()])
        
        checkpoint_path = output_dir + "emb_" + str(output_dim) + "/" + str(len(layer))
        checkpoint = ModelCheckpoint(checkpoint_path + "/model.h5",save_best_only=True, save_weights_only=False)
        model.fit(x=[users,products], y=ratings,shuffle = True, validation_split=0.1, batch_size=256, epochs=15, callbacks=[checkpoint])
        
        preds = model.predict([test_users, test_products])
        test_data['predictions'] = preds
        metrics = test_data.groupby(['user_id']).apply(calculate_metrics)
        rmse = np.sqrt(mean_squared_error(test_data['rating'], test_data['predictions']))
        avg_prec = metrics['precision'].mean()
        avg_recall = metrics['recall'].mean()
        avg_ndcg = metrics['ndcg'].mean()
        
        metrics_list.append([output_dim, len(layer),rmse, avg_prec, avg_recall, avg_ndcg])
        
        

2022-04-25 17:09:22.129900: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-25 17:09:27.361302: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9575 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:05:00.0, compute capability: 8.6
2022-04-25 17:09:27.362943: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 3652 MB memory:  -> device: 1, name: NVIDIA RTX A4000, pci bus id: 0000:08:00.0, compute capability: 8.6
2022-04-25 17:09:27.364201: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 wit

Epoch 1/15
  29/2814 [..............................] - ETA: 17s - loss: 14.0179 - root_mean_squared_error: 3.7376  

2022-04-25 17:09:30.975554: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


2814/2814 [==============================] - 23s 7ms/step - loss: 4.4891 - root_mean_squared_error: 1.7880 - val_loss: 2.4987 - val_root_mean_squared_error: 1.3412
Epoch 2/15
2814/2814 [==============================] - 20s 7ms/step - loss: 1.3577 - root_mean_squared_error: 0.9997 - val_loss: 1.2445 - val_root_mean_squared_error: 1.0377
Epoch 3/15
2814/2814 [==============================] - 20s 7ms/step - loss: 1.0267 - root_mean_squared_error: 0.9529 - val_loss: 1.1298 - val_root_mean_squared_error: 1.0139
Epoch 4/15
2814/2814 [==============================] - 16s 6ms/step - loss: 1.0031 - root_mean_squared_error: 0.9516 - val_loss: 1.1205 - val_root_mean_squared_error: 1.0126
Epoch 5/15
2814/2814 [==============================] - 20s 7ms/step - loss: 0.9989 - root_mean_squared_error: 0.9517 - val_loss: 1.1200 - val_root_mean_squared_error: 1.0132
Epoch 6/15
2814/2814 [==============================] - 20s 7ms/step - loss: 0.9965 - root_mean_squared_error: 0.9516 - val_loss: 1.1188

In [17]:
metrics_df = pd.DataFrame(metrics_list,columns = ['no of factors','no of hidden layers','rmse','precision','recall','ndcg'])

In [18]:
metrics_df.to_csv('all_metrics.csv')

In [2]:
model = load_model('models/emb_32/2/model.h5')

2022-04-26 16:27:52.689332: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-26 16:27:57.617195: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9575 MB memory:  -> device: 0, name: NVIDIA RTX A4000, pci bus id: 0000:05:00.0, compute capability: 8.6
2022-04-26 16:27:57.618783: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 3652 MB memory:  -> device: 1, name: NVIDIA RTX A4000, pci bus id: 0000:08:00.0, compute capability: 8.6
2022-04-26 16:27:57.620070: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:2 wit

In [3]:
model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 user_input (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 product_input (InputLayer)     [(None, 1)]          0           []                               
                                                                                                  
 embedding_30 (Embedding)       (None, 1, 32)        193280      ['user_input[0][0]']             
                                                                                                  
 embedding_31 (Embedding)       (None, 1, 32)        118592      ['product_input[0][0]']          
                                                                                            